In [42]:
%matplotlib inline

import os

import geopandas
import numpy as np
import pandas as pd
from shapely.geometry import Point
from shapely.geometry import LineString
import missingno as msn
import seaborn as sns
import matplotlib.pyplot as plt
import geohash

ModuleNotFoundError: No module named 'geohash'

In [16]:
# Read in all Atlantic hurricane data
hurricanes = pd.read_csv('../data/atlantic_hurricanes.csv')
hurricanes['Date'] = pd.to_datetime(hurricanes['Date'], errors='coerce')
hurricanes.dropna(inplace=True)
hurricanes['year'] = hurricanes['Date'].dt.year.astype(int)
hurricanes = hurricanes[hurricanes['Name'] != 'Unnamed']
hurricanes['slug'] = hurricanes['Name'] + '-' + hurricanes['year'].astype(str)
hurricanes['Long'] = 0 - hurricanes['Long']
hurricanes['coordinates'] = hurricanes[['Long', 'Lat']].values.tolist()
hurricanes['coordinates'] = hurricanes['coordinates'].apply(Point)

In [39]:
paths = hurricanes.groupby('slug')['coordinates'].apply(tuple).reset_index(name='path')
paths['len'] = paths['path'].apply(len)

In [41]:
for index, row in paths.iterrows():
    if row['len'] >= 100:
        pass
    else:
        line_coords = LineString(row['path'])
        geohashes = []
        for i in range(1, 101):
            partial = line_coords.interpolate(distance=i/100, normalized=True)
            geohashes.append(geohash)
        break

28
100
